In [11]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.optim import Adam
import torch.nn as nn
import pandas as pd
import os
from PIL import Image
import numpy as np

In [86]:
# Set the data paths
CELEBA_DATA_PATH = './Data/celeba'
IMG_PATH = os.path.join(CELEBA_DATA_PATH, 'img_align_celeba/img_align_celeba')
ATTR_PATH = os.path.join(CELEBA_DATA_PATH, 'list_attr_celeba.csv')
PARTITION_PATH = os.path.join(CELEBA_DATA_PATH, 'list_eval_partition.csv')
merged_path = "./Data/celeba/partitioned_multi_attr.csv"


def getImagePath(image_id):
    return os.path.join(IMG_PATH,image_id)

# Load the data|
attributes_df = pd.read_csv(ATTR_PATH)
partitioned_df = pd.read_csv(PARTITION_PATH)

# Calculate and sort the correlations
correlations = attributes_df.drop(columns=['image_id']).corrwith(attributes_df['Male']).abs().sort_values(ascending=False)

# Select attributes with high correlation and exclude subjective ones
selected_attributes = correlations[correlations > 0.2].index.difference(['Attractive', 'Chubby', 'High_Cheekbones'])

# Merge the DataFrames
merged_df = pd.merge(partitioned_df, attributes_df[['image_id'] + selected_attributes.tolist()], on='image_id')

# Convert to 0 and 1
merged_df[merged_df.select_dtypes(include=['number']).columns] = merged_df.select_dtypes(include=['number']).clip(lower=0)

male_column = merged_df.pop('Male')  # remove Male column and store it
merged_df.insert(1, 'Male', male_column)  # insert Male column after image_id

# Export 
merged_df.to_csv("./Data/celeba/partitioned_multi_attr.csv", index=False)

# Display the first rows of the merged DataFrame
merged_df.head()

,image_id,Male,partition,5_o_Clock_Shadow,Arched_Eyebrows,Bags_Under_Eyes,Big_Nose,Blond_Hair,Bushy_Eyebrows,Double_Chin,...,No_Beard,Pointy_Nose,Rosy_Cheeks,Sideburns,Wavy_Hair,Wearing_Earrings,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,1,1,0,0,1
1,000002.jpg,0,0,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,000003.jpg,1,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1
3,000004.jpg,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,1,1,0,1
4,000005.jpg,0,0,0,1,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,1


In [100]:
from torch.utils.data import Dataset
from PIL import Image
import os

class CelebADataset(Dataset):
    def __init__(self, file_paths, file_to_label, transform=None):
        self.file_paths = file_paths
        self.file_to_label = file_to_label
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_name = self.file_paths[idx]
        image = Image.open(img_name).convert('RGB')
        label = self.file_to_label[os.path.basename(img_name)][0]  # Only the "Male" label
        attributes = self.file_to_label[os.path.basename(img_name)][2:]  # After partition rest are attributes
        if self.transform:
            image = self.transform(image)

        return image, label, torch.tensor(attributes, dtype=torch.float32)


    
df = pd.read_csv("./Data/celeba/partitioned_multi_attr.csv")
train_df = df[df['partition'] == 0]
val_df = df[df['partition'] == 1]
test_df = df[df['partition'] == 2]

df_labels = df.set_index('image_id')
filename_to_label = {filename: labels.values for filename, labels in df_labels.iterrows()}
file_paths = df['image_id'].apply(getImagePath).tolist()


# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# train_dataset = CelebADataset(file_paths, filename_to_label, transform=transform)
# val_dataset = CelebADataset


# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)

In [101]:
# Assuming the images are in a directory named 'images' in the current working directory
# Create separate file path and label mappings for each dataset partition
train_file_paths = train_df['image_id'].apply(getImagePath).tolist()
val_file_paths = val_df['image_id'].apply(getImagePath).tolist()
test_file_paths = test_df['image_id'].apply(getImagePath).tolist()

train_filename_to_label = {filename: labels.values for filename, labels in train_df.set_index('image_id').iterrows()}
val_filename_to_label = {filename: labels.values for filename, labels in val_df.set_index('image_id').iterrows()}
test_filename_to_label = {filename: labels.values for filename, labels in test_df.set_index('image_id').iterrows()}

# Initialize the datasets for each partition
train_dataset = CelebADataset(train_file_paths, train_filename_to_label, transform=transform)
val_dataset = CelebADataset(val_file_paths, val_filename_to_label, transform=transform)
test_dataset = CelebADataset(test_file_paths, test_filename_to_label, transform=transform)

# Create data loaders for each dataset partition
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [102]:
# Load a pre-trained ResNet model
model = resnet50(pretrained=True)

# Modify the model for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Male/Female

/Users/kohjunkai/anaconda3/envs/gender_classification/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kohjunkai/anaconda3/envs/gender_classification/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [103]:
from torchvision.models import resnet50
import torch.nn as nn

# Load a pre-trained ResNet model
model = resnet50(pretrained=True)

# Assume the number of attributes is equal to the number of columns minus 3 (for the image_id, male, partition)
num_attributes = merged_df.shape[1] - 3

# Modify the model for binary classification plus additional attributes
class MultiInputResNet(nn.Module):
    def __init__(self, base_model, num_attributes, num_classes=2):
        super().__init__()
        self.base_model = base_model
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()  # Remove the original fully connected layer

        # Fully connected layer for attributes
        self.attr_fc = nn.Linear(num_attributes, 224)  

        # Final fully connected layer that takes both image features and attributes
        self.final_fc = nn.Linear(num_ftrs + 224, 2)  # Modify num_classes based on your classification problem

    def forward(self, image, attributes):
        # Get image features from the ResNet
        img_features = self.base_model(image)

        # Process attributes
        attr_features = self.attr_fc(attributes)

        # Concatenate image and attribute features
        combined_features = torch.cat((img_features, attr_features), dim=1)

        # Final classification layer
        return self.final_fc(combined_features)

# Update the model
multi_attr_model = MultiInputResNet(base_model=model, num_attributes=num_attributes)


In [104]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, checkpoint_name="muti_attr_checkpoint.pt"):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            checkpoint_name (str): Name of the checkpoint file. 
                            Default: "checkpoint.pt"
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.checkpoint_name = checkpoint_name 

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decreases."""
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.checkpoint_name) 
        self.val_loss_min = val_loss

        
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import torch

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, checkpoint_name):
    # Initialize early stopping
    early_stopping = EarlyStopping(patience=5, verbose=True, checkpoint_name=checkpoint_name)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_pbar = tqdm(train_loader, unit="batch")

        for images, labels, attributes in train_pbar:
            train_pbar.set_description(f"Epoch {epoch+1}/{num_epochs}")
            images, labels, attributes = images.to(device), labels.to(device).float(), attributes.to(device).float()
            
            optimizer.zero_grad()
            outputs = model(images, attributes)
            
            labels = labels.view(-1, 1) if labels.ndim == 1 else labels
            
            loss = criterion(outputs, labels)  
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            train_pbar.set_postfix(loss=loss.item())

        train_loss = running_loss / len(train_loader.dataset)

        model.eval()
        val_running_loss = 0.0
        val_pbar = tqdm(val_loader, unit="batch")

        with torch.no_grad():
            for images, labels, attributes in val_pbar:
                val_pbar.set_description(f"Val Epoch {epoch+1}/{num_epochs}")
                images, labels, attributes = images.to(device), labels.to(device).float(), attributes.to(device).float()
                
                outputs = model(images, attributes)
                
                labels = labels.view(-1, 1) if labels.ndim == 1 else labels
                
                loss = criterion(outputs, labels)  
                val_running_loss += loss.item() * images.size(0)
                val_pbar.set_postfix(loss=loss.item())

        val_loss = val_running_loss / len(val_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs} Train loss: {train_loss:.4f} Val loss: {val_loss:.4f}')

        # Call early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        scheduler.step(val_loss)

    model.load_state_dict(torch.load(checkpoint_name))
    return model


In [105]:
import torch
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torchvision.models import resnet50
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# Assuming the `MultiInputResNet` class has been defined as discussed previously

# Check if GPU is available and move the model to GPU
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assuming train_loader and val_loader have been defined and are ready to use
# Define the loss function and optimizer
criterion = BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.1, verbose=True)

# Define the number of epochs
num_epochs = 10  # Change this to your desired number of epochs

# Run the training function
trained_model = train_model(
    model=multi_attr_model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler, 
    num_epochs=num_epochs, 
    device=device, 
    checkpoint_name="/weights/multi_attr_checkpoint.pt"
)

# Save the trained model state
torch.save(trained_model.state_dict(), "/weights/multi_attr_model_final.pt")

print("Training complete.")


Epoch 1/10:   0%|                                   | 0/5087 [00:04<?, ?batch/s]


ValueError: Target size (torch.Size([32, 1])) must be the same as input size (torch.Size([32, 2]))